In [26]:
import os
from google.cloud import bigquery
from datetime import datetime
import pandas_gbq
import pandas as pd
import random
import numpy as np

In [27]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Alfred/Documents/Trabajo/Personal/los-outliers/Data/climate-prediction-389419-10c7097dc497.json"


In [28]:

def upload_dataframe_to_bigquery(dataframe, full_table_id):
    # Create a BigQuery client
    client = bigquery.Client()
    # Append the dataframe to the BigQuery table
    pandas_gbq.to_gbq(dataframe, full_table_id, if_exists='append', project_id='climate-prediction-389419')


In [29]:
# clean data function
def clean_data(file_path):
    df = pd.read_csv(file_path)
    df = df[['Year', 'Start Month', 'Country', 'Location', 'Disaster Type']]

    # Rename 'Start Month' column to 'Month'
    df.rename(columns={'Start Month': 'Month'}, inplace=True)

    # Create column with Precipitation (MM) random values between 0 and 100
    df['Precipitation_MM'] = df['Year'].apply(lambda x: random.randint(0, 100))

    # Create column with Temperature random values between -10 and 40
    df['Temperature'] = df['Year'].apply(lambda x: random.randint(-10, 40))

    # Create Column Disaster_Probability with random values between 0 and 1
    df['Disaster_Probability'] = df['Year'].apply(lambda x: random.random())

    # Drop null values in Location and Month
    df = df.dropna(subset=['Location', 'Month'])

    # Convert 'Year' to int
    df['Year'] = df['Year'].astype(int)

    # Create column with Disaster Probability (or prediction) random values between 0 and 1
    df['Disaster_Probability'] = df['Year'].apply(lambda x: random.random())

    # Replace spaces in column names with underscores
    df.columns = df.columns.str.replace(' ', '_')

    return df


In [33]:
df = clean_data('/Users/Alfred/Documents/Trabajo/Personal/los-outliers/Data/1900_2021_DISASTERS.xlsx - emdat data.csv')

In [34]:
df.head(10)

,Year,Month,Country,Location,Disaster_Type,Precipitation_MM,Temperature,Disaster_Probability
2,1902,4.0,Guatemala,"Quezaltenango, San Marcos",Earthquake,44,10,0.698595
5,1903,4.0,Canada,"Frank, Alberta",Mass movement (dry),37,0,0.163643
7,1904,11.0,Bangladesh,Chittagong,Storm,10,11,0.941331
8,1905,8.0,Canada,"Spence's Bridge, British Columbia",Mass movement (dry),100,33,0.010433
9,1905,4.0,India,Kangra,Earthquake,52,1,0.729428
10,1906,8.0,Chile,Valparaiso,Earthquake,16,15,0.708363
11,1906,1.0,Colombia,Tumako,Earthquake,44,22,0.201121
12,1906,5.0,Belgium,Louvain region,Flood,57,12,0.496553
15,1907,10.0,China,Tien Shan,Earthquake,80,6,0.447261
17,1908,4.0,Canada,"Notre Dame de La Salette, Quebec",Mass movement (dry),90,35,0.571201


In [35]:
# read csv file and save as dataframe
extended_df = pd.read_csv('/Users/Alfred/Documents/Trabajo/Personal/los-outliers/Data/extendido_1900_2021_DISASTERS.xlsx - emdat data.csv')
# rename precipitation column to Precipitation_MM
extended_df.rename(columns={'Precipitation_(MM)': 'Precipitation_MM'}, inplace=True)

# rename Disaster_Probability_(or_prediction) column to Disaster_Probability
extended_df.rename(columns={'Disaster_Probability_(or_prediction)': 'Disaster_Probability'}, inplace=True)


In [36]:
# replace Disaster_Probability values with random values between 0 and 1
extended_df['Disaster_Probability'] = extended_df['Disaster_Probability'].apply(lambda x: random.random())

# Temperature under 5 is cold, between 5 and 25 is warm, and above 25 is hot
extended_df['Temperature'] = extended_df['Temperature'].apply(lambda x: 'cold' if x < 5 else 'warm' if 5 <= x <= 25 else 'hot')

# Precipitation under 10 is low, between 10 and 30 is medium, and above 30 is high
extended_df['Precipitation_MM'] = extended_df['Precipitation_MM'].apply(lambda x: 'low' if x < 10 else 'medium' if 10 <= x <= 30 else 'high')

In [37]:
extended_df.head()

,Year,Country,Location,Month,Disaster_Type,Precipitation_MM,Temperature,Disaster_Probability
0,1902,Guatemala,"Quezaltenango, San Marcos",4.0,Earthquake,high,hot,0.122514
1,1902,Guatemala,NaN,4.0,Volcanic activity,high,warm,0.122930
2,1902,Guatemala,NaN,10.0,Volcanic activity,high,cold,0.780375
3,1903,Canada,"Frank, Alberta",4.0,Mass movement (dry),high,cold,0.788110
4,1904,Bangladesh,Chittagong,11.0,Storm,high,hot,0.327987


In [39]:
upload_dataframe_to_bigquery(extended_df, 'climate-prediction-389419.climate_prediction.disasters')